# Introduction to `sns_modeling` tool

The separation network synthesis (sns) tool provides an initial design for distillation separation systems using thermally coupled distillation columns. The intent of the tool is to determine cost optimal separation network structure for process synthesis and intensification applications. This notebook provides an overview of how to install and use the package to build and solve separation network synthesis problems.

The main goals of the tool are to aid a designer in determining:

1. Selection of component splits in a separation network
2. Process topology with an emphasis on heat integration
3. Initial sizing and costing of unit operations

## Table of Contents:

1. [Package Setup and Installation](##1-package-setup-and-installation)
2. [Package Structure](##2-package-structure)
3. [Problem Setup and Data Loading](##3-problem-setup-and-data-loading)
4. [Problem Scaling and Transformation](##4-problem-scaling-and-transformation)
5. [Problem Solution](##5-problem-solution)
6. [Solution Output and Interpretation](##6-solution-output-and-interpretation)

## 1. Package Setup and Installation

Download or clone the package from the Github repo [sns_modeling](https://github.com/pfauk/sns_modeling). In a terminal, navigate to the directory location and install the package dependencies by running: 

```
pip install -r requirements.txt
```

**Important**: this package requires an installed version of Gurobi to solve the MIQCP model. It is possible to pip install Gurobi from the Python Package Index (PyPI). However, the free version comes with a trial license that will only be able to solve models of a smaller size (2,000 variables or constraints).

After installing dependencies, navigate to the directory of the package and run:

```
pip install -e . 
```

## 2. Package Structure

The package has several main components that contribute to the overall functionality of the design tool. The main components are:

1. Model generation
2. Superstrucutre generation
3. Data directory
4. Utilities
5. Documentation

**1. Model generation**

This tool builds a mathematical program as a Pyomo Concrete Model. The Pyomo Concrete Model object is generated through a function call to the `build_model` function from `src\thermal_coupled\therm_dist.py`. All of the mathematical modeling components are defined and documented in this script. This function is the core functionality of the package.

**2. Superstructure generation**

An important aspect of process synthesis modeling is defining a process superstructure. This package includes functionality to automatically generate a network superstructure and pass it to the `build_model` function. The `src\superstructure` directory contains the functionality to build the state task network (stn) for the superstructure of the problem.

**3. Data directory**

Data for this problem should be structured in the format of the provided example excel spreadsheets in `src\data`. Users do not have to place data files in that directory, but it is the default location for the utility functions to pull data from. 

**4. Utilities**

The `utils.py` script contains functionality for loading data from excel spreadsheets into objects that can be passed to the `build_model` function. There are utility functions for printing and saving models and the resulting solutions.

**5. Documentation**

The `docs` directory contains files that give a detailed explanation of the mathematical model, empirical correlations that were used, and superstructure definition.

## 3. Problem Setup and Data Loading

Here we will show how to use the modeling tool's core functionality to build and solve a process synthesis problem. The general problem that we want to solve is: 

*Given an N component zeotropic mixture, determine the cost optimal separation sequence, column design, and heat integration to separate the mixture into N components*.

This can be generally visualized as separating some mixture of components {A, B, C, D} into streams of relatively high purity.


<img src="images/problem_statement.png" alt="Problem Statement" width="550" height="250" />

*Representation of the conceptual problem of separating a 4 component mixture into 4 high purity product streams*

The workflow for this design problem can be outlined as:

1. Define and construct a process superstructure
2. Define all relevant species and system data
3. Build the generalized disjunctive program (GDP)
4. Transform the GDP into a mixed integer quadratically constrained program (MIQCP)
5. Solve the mathematical program

### 3 a. Process Superstructure

We first need to represent the overall superstructure of a distillation process. The superstructure represents the solution space to the problem, so the representation has to be sufficiently detailed to provide a meaningful solution representation. We include functionality to automatically build superstructures for an arbitrary number of components in a mixture. There are two different types of distillation superstructures we can build, those with separation splits between **consecutive key components** and those with splits between **non-consecutive key components**.

For a single distillation column, we specify light key and heavy components that we want to separate out in high purity in the distillate and bottoms respectively. This is referred to as the split of the components in a mixture. There are two options for how to specify the key components. We can specify a mixture of arbitrary components that are ordered by decreasing relative volatility as: *ABC*.

For the case of splits between consecutive key components, there are two possible split options for this mixture: *A/BC* and *AB/C*. In the example of the split *A/BC*, *A* is the light key component and *B* is the heavy key component.

For the case of splits between non-consecutive key components, there are three possible split options for this mixture: *A/BC*, *AB/BC* *AB/C*. In the example of the split *AB/BC*, *A* is the light key component, *C* is the heavy key component, and *B* is an intermediate component that is allowed to distribute between the distillate and the bottoms of a distillation column.

We will now show the first step of the modeling approach, which is build the problem structure as a state-task network (STN).

In [1]:
## Imports
import logging
import os
import pyomo.environ as pyo
from pyomo.util.infeasible import log_infeasible_constraints, find_infeasible_constraints
from pyomo.util.model_size import build_model_size_report
from idaes.core.util.model_statistics import report_statistics
from utils import (
    Data,
    get_model_type,
    pprint_network,
    pprint_tasks,
    save_model_to_file,
    save_solution_to_file,
    get_model_type,
    print_constraint_type)
from superstructure.stn import stn
from superstructure.stn_nonconsecutive import stn_nonconsecutive
from thermal_coupled.therm_dist import build_model

In [2]:
# specify number of components in the feed to the system
n = 4

We choose the example of a 4 component system feed and a superstructure that uses splits only between consecutive key components.

In [3]:
# build state-task network superstrucutre and associated index sets
network_superstructure = stn(n)
network_superstructure.generate_tree()
network_superstructure.generate_index_sets()

This state-task network superstrucutre can be visually represented as a graph

<img src="images/consecutive_split_stn.png" alt="4 Component STN" width="975" height="500">

*The state-task network superstrucutre for a 4 component mixture with splits between consecutive key components*

In [4]:
# the instantiated network_superstructure graph does include text display
network_superstructure.print_tree()

State(ABCD)
  Task(A/BCD)
    State(A, final=True)
    State(BCD)
      Task(B/CD)
        State(B, final=True)
        State(CD)
          Task(C/D)
            State(C, final=True)
            State(D, final=True)
      Task(BC/D)
        State(BC)
          Task(B/C)
            State(B, final=True)
            State(C, final=True)
        State(D, final=True)
  Task(AB/CD)
    State(AB)
      Task(A/B)
        State(A, final=True)
        State(B, final=True)
    State(CD)
      Task(C/D)
        State(C, final=True)
        State(D, final=True)
  Task(ABC/D)
    State(ABC)
      Task(A/BC)
        State(A, final=True)
        State(BC)
          Task(B/C)
            State(B, final=True)
            State(C, final=True)
      Task(AB/C)
        State(AB)
          Task(A/B)
            State(A, final=True)
            State(B, final=True)
        State(C, final=True)
    State(D, final=True)


There is the alternative use of a superstructure that uses splits between non-consecutive key components. An example for such a network with a feed of 4 components is shown below. Note that this network structure has more nodes and more connections, which will result in a more difficult optimization problem.

<img src="images/nonconsecutive_split_stn.png" alt="4 Component STN" width="950" height="500">

*The state-task network superstrucutre for a 4 component mixture with splits between non-consecutive key components*

### 3 b. Data

After building a superstructure, a user needs to provide parameters to define a specific problem instance. Data needs to be provided for both the overall system and for the species present in the feed mixture. The `Data` class from the tool utilities is provided to parse data from a spreadsheet and construct an object to pass to the `build_model` function. Example data spreadsheet files are included and users should utilize this same formatting for new problem instances. The `src/data` directory is the default location for the `Data` class to located files, but alternative file locations can be provided as a key word argument.

In [5]:
data_file = os.path.join('notebook_examples', '4_comp_linear_hydrocarbons.xlsx')

# import problem data for system and relevant species to data object
mixture_data = Data(data_file)

**Species parameters**

This sheet contains parameters for each chemical species in the system. Species index should just be upper case capital letters: A, B, C, D. Species should be ordered by decreasing relative volatility, with A as the most volatile and the last species as the least volatile.

Inlet fractions are mole fractions. Enure they sum to 1.

Relative volatilities can be determined from ASPEN properties for binary mixtures relative to the least volatile species in the system. The relative volatility ($\alpha_i$) should be found in ASPEN for the temperature and pressure specified in the system sheet. The system is modeled so that the feeds are liquids at bubble point. The liquid density of the species at the system temperature and pressure is used to do empirical correlations for equipment sizing.

The recovery ($x_i$) for each component is the fraction of the molar flow in the outlet of that species stream, relative to the total inlet to the system. Note that a product recovery constraint is different than a product purity constraint. Setting the recoveries too high (such as setting all the recoveries to values of 1) may lead to an infeasible problem, as such a separation could require columns with an infinite number of stages. 

The given example problem is the separation of a hydrocarbon mixture. The species data in the sheet can be viewed as:

<img src="images/species_data.png" alt="4 Component Species Data" width="1200" height="250">

In [6]:
# print out the feed species data 
print('Mixture species data')
print('================================================================')
print(mixture_data.species_df)

Mixture species data
     Species index  Inlet Mole Frac  Relative Volatility  \
0   n-Hexane     A             0.10                 5.74   
1  n-Heptane     B             0.25                 3.68   
2  n-Octance     C             0.40                 2.06   
3   n-Nonane     D             0.25                 1.00   

   Liquid Density [kg/m^3]  Molecular Weight  \
0                      877                86   
1                      868               100   
2                      868               114   
3                      883               128   

   Enthalpy of Vaporization [kJ/mol]  Recovery  
0                              28.87      0.97  
1                              31.71      0.97  
2                              34.43      0.97  
3                              36.93      0.97  


Similarly, the system data in the `4_comp_linear_hydrocarbons.xlsx` sheet for this example can be viewed:

<img src="images/system_data.png" alt="4 Component Species Data">

In [7]:
# print out the system data 
print('System data')
print('================================================================')
print(mixture_data.system_df)

System data
   F0 [kmol/hr]  Pressure [bar]  Temp [C]  Cost cooling [$/kJ]  \
0           300               1        85              0.00019   

   Cost heating [$/kJ]  
0              0.00509  


## 4. Problem Scaling and Transformation

We now build the actual mathematical model as a Pyomo Concrete Model using the `build_model` function. This function will return both a Pyomo model and a dictionary of scaling factors. The key word arg `scale` allows the users to apply pre-defined scaling factors that scale down cost coefficients in the model. The down-scaling of the model helps to reduce the size of the feasible space for the problem and aid in speeding up computational solution time. For small problem instances, scaling does not provide as much computational benefit, but as the problem to be solved grows with the number of components to separate, scaling becomes increasingly important to obtain a solution in a reasonable time window. 

In [8]:
# building Pyomo model
model, scaling_factors = build_model(network_superstructure, mixture_data, scale=True)

It can sometimes be useful to inspect the model object prior to solution. The model that is constructed by the `build_model` function is a generalized disjunctive program (GDP). The `get_model_type` from `utils.py` allows you to see what type of mathematical model the Pyomo model object contains. Furthermore, the `save_model_to_file` function can be used to create a text file to inspect the entire model object in a pretty printed format. Saving the model for inspection is best done *prior to transforming of the GDP model*. By default, the pretty printed Pyomo model is saved to `thermal_coupled\saved_models`.

In [9]:
# saving the pyomo model to a file
save_model_to_file(model, '4_comp_model')

In [10]:
# check the model type prior to transformation
print(f'Model type before transformation: {get_model_type(model)}')

Model type before transformation: GDP


Finally, we need to transform the generalized disjunctive program to a mixed integer program to apply solvers to the model. This is done using Pyomo.gdp's `TransformationFactory` as shown below. The GDP model also contains logical constraints that need to be transformed prior to passing the model to a solver.

In [11]:
# use of Pyomo.gdp to apply Big-M transformation
pyo.TransformationFactory('core.logical_to_linear').apply_to(model)

mbigm = pyo.TransformationFactory('gdp.bigm')
mbigm.apply_to(model)

print(f'Model type after transformation: {get_model_type(model)}')

Model type after transformation: MIQCP


## 5. Problem Solution

After applying the Big-M transformation using Pyomo.GDP's TransformationFactory, the Pyomo model is a non-convex mixed-integer quadratically constrained program (MIQCP). We can use Gurobi to solve the model. Gurobi has a number of parameters that can be passed to the solver. A [full list of solver parameters](https://www.gurobi.com/documentation/current/refman/parameters.html) can be founds on the Gurobi website. For now we recommend setting the NumericFocus and nonConvex parameters to values of 2. Experience with the problem has also shown that a reduced MIPGap value can aid in solution time without negatively impacting the quality of the resulting solution. Gurobi's default MIPGap setting is $1e-4$. For more complex problems, a large amount of time can be spent by the solver in reducing the MIPGap below 0.1% but producing a very similar solution.

In [12]:
# Pyomo solver factory
solver = pyo.SolverFactory('gurobi')

# Gurobi solver options
solver.options = {'NumericFocus': 2,
                  'nonConvex': 2,
                  'MIPgap': 1e-4}

Now send the Pyomo model to the solver

In [13]:
results = solver.solve(model, tee=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-24
Read LP format model from file C:\Users\kevin\AppData\Local\Temp\tmpfpd6abl9.pyomo.lp
Reading time = 0.03 seconds
x1: 1168 rows, 488 columns, 2995 nonzeros
Set parameter NumericFocus to value 2
Set parameter NonConvex to value 2
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1168 rows, 488 columns and 2995 nonzeros
Model fingerprint: 0x062da0d4
Model has 424 quadratic constraints
Variable types: 438 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [6e-03, 1e+07]
  QMatrix range    [5e-05, 4e+01]
  QLMatrix range   [8e-02, 6e+10]
  Objective range  [9e-02, 1e+00]
  Bounds range     [2e-01, 1e+07]
  RHS range        [1e+00, 1e+07]
  QRHS range       [5e-03

## 6. Solution Output and Interpretation

To see the output of the solution, use the `pprint_network` function from the package utilities to inspect the overall network design

In [14]:
pprint_network(model)


NETWORK OVERVIEW
4 Component GDP Distillation Model
*All Flow Units in [kmol/hr]*

System Objective: $1,340,041.86

CAPEX: $6,039,029.41
OPEX: $803,610.38

System Feed Total: ABCD (300.0 [kmol/hr])
Feed A 30.0
Feed B 75.0
Feed C 120.0
Feed D 75.0

Active Separation Tasks
A/BCD
B/CD
C/D

Column A/BCD:
Column cost $2,891,056.82
Number of trays: 32.0
Column height: 65.4 [m]
Column diameter: 4.0 [m^2]
Feed flow (FT): 300.0000
Distillate (DT): 31.3500
Bottoms (BT): 268.6500

Column B/CD:
Column cost $2,744,189.95
Number of trays: 24.0
Column height: 54.8 [m]
Column diameter: 4.3 [m^2]
Feed flow (FT): 268.6500
Distillate (DT): 75.0675
Bottoms (BT): 193.5825

Column C/D:
Column cost $224,596.88
Number of trays: 20.0
Column height: 17.7 [m]
Column diameter: 2.0 [m^2]
Feed flow (FT): 193.5825
Distillate (DT): 117.3405
Bottoms (BT): 76.2420

DETAILED NETWORK VIEW


Separation task A/BCD is active

Total Flows for Column A/BCD [kmol/hr]:
Feed flow (FT): 300.0000
Distillate (DT): 31.3500
Bottoms 

You can save the pretty printed solution output to a text file with the used of `save_solution_to_file`. The `thermal_coupled\results` directory is the default save location.

In [15]:
save_solution_to_file(model, '4_comp_solution')

Below is a visual representation of the above solution for a separation network for the given 4 component hydrocarbon feed.

<img src="images/4_comp_hydrocarbon_solution.png" alt="4 Component Species Solution">

We wil show one more solution, but using the non-consecutive split superstructure to highlight some of the differences in the solution space. Below are all of the steps needed to build and solve the problem in a single code cell. 

In [16]:
# specify number of components in the feed to the system
num_components = 3

# build state-task network superstrucutre and associated index sets
network_superstructure_nonconsecutive = stn_nonconsecutive(num_components)
network_superstructure_nonconsecutive.generate_tree()
network_superstructure_nonconsecutive.generate_index_sets()


data_file_3_comp = os.path.join('notebook_examples', '3_comp_linear_hydrocarbons.xlsx')

# import problem data for system and relevant species to data object
mixture_data_3_comp = Data(data_file_3_comp)

# building Pyomo model
model_nonconsecutive, scaling_factors = build_model(network_superstructure_nonconsecutive, mixture_data_3_comp, scale=True)

# use of Pyomo.gdp to apply Big-M transformation
pyo.TransformationFactory('core.logical_to_linear').apply_to(model_nonconsecutive)

mbigm = pyo.TransformationFactory('gdp.bigm')
mbigm.apply_to(model_nonconsecutive)

# Gurobi solver options
solver.options = {'NumericFocus': 2,
                  'nonConvex': 2,
                  'MIPgap': 1e-4}

results_nonconsecutive = solver.solve(model_nonconsecutive, tee=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-24
Read LP format model from file C:\Users\kevin\AppData\Local\Temp\tmpov9iu_xv.pyomo.lp
Reading time = 0.03 seconds
x1: 537 rows, 218 columns, 1311 nonzeros
Set parameter NumericFocus to value 2
Set parameter NonConvex to value 2
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 537 rows, 218 columns and 1311 nonzeros
Model fingerprint: 0x480394e9
Model has 172 quadratic constraints
Variable types: 189 continuous, 29 integer (29 binary)
Coefficient statistics:
  Matrix range     [6e-03, 1e+07]
  QMatrix range    [5e-05, 3e+01]
  QLMatrix range   [8e-02, 7e+10]
  Objective range  [9e-02, 1e+00]
  Bounds range     [6e-01, 1e+07]
  RHS range        [1e+00, 1e+07]
  QRHS range       [1e-02, 

In [17]:
pprint_network(model_nonconsecutive)


NETWORK OVERVIEW
3 Component GDP Distillation Model
*All Flow Units in [kmol/hr]*

System Objective: $823,025.89

CAPEX: $1,547,722.43
OPEX: $685,545.67

System Feed Total: ABC (350.0 [kmol/hr])
Feed A 105.0
Feed B 105.0
Feed C 140.0

Active Separation Tasks
A/B
AB/BC
B/C

Column A/B:
Column cost $315,141.15
Number of trays: 27.0
Column height: 24.7 [m]
Column diameter: 2.1 [m^2]
Feed flow (FT): 124.5160
Distillate (DT): 95.6508
Bottoms (BT): 28.8652

Column AB/BC:
Column cost $847,033.69
Number of trays: 12.0
Column height: 21.7 [m]
Column diameter: 3.7 [m^2]
Feed flow (FT): 350.0000
Distillate (DT): 124.5160
Bottoms (BT): 225.4840

Column B/C:
Column cost $280,915.64
Number of trays: 21.0
Column height: 19.9 [m]
Column diameter: 2.2 [m^2]
Feed flow (FT): 225.4840
Distillate (DT): 94.7723
Bottoms (BT): 130.7117

DETAILED NETWORK VIEW


Separation task A/B is active

Total Flows for Column A/B [kmol/hr]:
Feed flow (FT): 124.5160
Distillate (DT): 95.6508
Bottoms (BT): 28.8652

Componen

Below is a visual representation of solution. Note that the non-consecutive split *AB/BC* is selected for the initial separation task. The first column in the network functions as a prefractionator, which is a common design for more complex distillation systems. Further more, *B* does not have an associated final product heat exchanger, meaning that it is taken off as a side draw. Due to their similar diameter, the second and third columns could be constructed as a single column in which a side product is taken off.

<img src="images/3_comp_hydrocarbon_solution.png" alt="3 Component Species Solution">

*References:*

1) – Caballero, J. A., & Grossmann, I. E. (2004). Design of distillation sequences: From conventional to fully thermally coupled distillation systems. Computers & Chemical Engineering

2) – Caballero, J. A., & Grossmann, I. E. (2001). Generalized Disjunctive Programming Model for the Optimal Synthesis of Thermally Linked Distillation Columns. Industrial & Engineering Chemistry Research

